In [ ]:
import boto3
import json
import datetime
from IPython.display import display, JSON

bedrock_agent = boto3.client("bedrock-agent", region_name="us-east-1")
bedrock_runtime = boto3.client("bedrock-runtime", region_name="us-east-1")
bedrock_agent_runtime = boto3.client("bedrock-agent-runtime", region_name="us-east-1")
model_id = "amazon.nova-micro-v1:0"

In [34]:
prompt_name = f"job-description-{datetime.datetime.now().strftime('%Y%m%d%H%M%S')}"

template_text = '''
You are an HR assistant.

Write a professional, inclusive job description using the following inputs:

Job title: {{job_title}}
Responsibilities: {{responsibilities}}
Requirements: {{requirements}}
Location: {{location}}
Work type: {{work_type}}

- Start with a clear summary
- Use concise, inclusive language
- Keep it under 250 words
'''
response = bedrock_agent.create_prompt(
    name=prompt_name,
    description="Generates inclusive job descriptions from structured inputs",
    defaultVariant="v1",
    variants=[
        {
            "name": "v1",
            "modelId": model_id,
            "templateType": "TEXT",
            "templateConfiguration": {
                "text": {
                    "inputVariables": [
                        {"name": "job_title"},
                        {"name": "responsibilities"},
                        {"name": "requirements"},
                        {"name": "location"},
                        {"name": "work_type"}
                    ],
                    "text": template_text
                }
            },
            "inferenceConfiguration": {
                "text": {
                    "maxTokens": 500,
                    "temperature": 0.7,
                    "topP": 0.9,
                    "stopSequences": []
                }
            }
        }
    ]
)
print("\n==================== Response Object ====================\n")
display(JSON(response))

print("\n==================== Prompt ARN ====================\n")
print(response['arn'])


==================== Response Object ====================



<IPython.core.display.JSON object>


==================== Prompt ARN ====================

arn:aws:bedrock:us-east-1:982248023567:prompt/ARZ39REHBM


In [35]:
response = bedrock_agent.create_prompt_version(
    description='Initial prompt for creating job description documents.',
    promptIdentifier="arn:aws:bedrock:us-east-1:982248023567:prompt/MI1S36L51W"
)

print("\n==================== PROMPT VERSION ARN ====================\n")
print(response["arn"])


==================== PROMPT VERSION ARN ====================

arn:aws:bedrock:us-east-1:982248023567:prompt/MI1S36L51W:2


In [56]:
prompt_arn_with_version ="arn:aws:bedrock:us-east-1:982248023567:prompt/MI1S36L51W:2"

response = bedrock_runtime.converse(
    modelId=prompt_arn_with_version,
    promptVariables={
        "job_title": {"text": "UX Designer"},
        "responsibilities": {"text": "Design user interfaces, run usability testing, collaborate with product teams"},
        "requirements": {"text": "3+ years experience, Figma, HTML/CSS knowledge, communication skills"},
        "location": {"text": "New York or remote"},
        "work_type": {"text": "Full-time"}
    }
)

print("\n==================== Response Text ====================\n")
print(response['output']['message']['content'][0]['text'])


==================== Response Text ====================

**Job Title: UX Designer**

**Summary:**
We are seeking a talented and inclusive UX Designer to join our dynamic team. As a UX Designer, you will play a vital role in designing intuitive user interfaces, conducting usability testing, and collaborating closely with our product teams to create exceptional user experiences.

**Responsibilities:**
- Design compelling and user-friendly interfaces.
- Conduct thorough usability testing to refine and enhance user experiences.
- Collaborate with cross-functional teams to ensure seamless product development.

**Requirements:**
- 3+ years of experience in UX design.
- Proficiency in Figma for design prototyping.
- Strong knowledge of HTML/CSS for implementing designs.
- Excellent communication skills to articulate ideas and feedback.

**Location:**
This position is available in New York or as a remote opportunity, allowing for a flexible and inclusive work environment.

**Work Type:**
Full

In [57]:
def handle_response_stream(response):
    try:
        event_stream = response['optimizedPrompt']
        for event in event_stream:
            if 'optimizedPromptEvent' in event:
                print("\n==================== OPTIMIZED PROMPT ====================\n")
                print(event['optimizedPromptEvent']['optimizedPrompt']['textPrompt']['text'])
    except Exception as e:
        raise e


prompt_input = {
    "textPrompt": {
        "text": template_text
    }
}

response = bedrock_agent_runtime.optimize_prompt(
            input=prompt_input,
            targetModelId=model_id
        )

print("\n==================== ORIGINAL PROMPT ====================\n")
print(template_text)
handle_response_stream(response)


==================== ORIGINAL PROMPT ====================


You are an HR assistant.

Write a professional, inclusive job description using the following inputs:

Job title: {{job_title}}
Responsibilities: {{responsibilities}}
Requirements: {{requirements}}
Location: {{location}}
Work type: {{work_type}}

- Start with a clear summary
- Use concise, inclusive language
- Keep it under 250 words


==================== OPTIMIZED PROMPT ====================

"# HR Assistant: Job Description Generator\n\n## Task\nCreate a professional, inclusive job description based on the provided information.\n\n## Input Information\n### Job Title\n{{job_title}}\n\n### Responsibilities\n{{responsibilities}}\n\n### Requirements\n{{requirements}}\n\n### Location\n{{location}}\n\n### Work Type\n{{work_type}}\n\n## Output Requirements\n1. Begin with a clear, compelling summary of the position (1-2 sentences)\n2. Structure the job description with appropriate headings (About the Role, Responsibilities, Requir

In [58]:
prompt_identifier = "arn:aws:bedrock:us-east-1:982248023567:prompt/ARZ39REHBM"

existing_prompt = bedrock_agent.get_prompt(promptIdentifier=prompt_identifier)
print("\n==================== ORIGINAL PROMPT ====================\n")
print(existing_prompt["variants"][0]["templateConfiguration"]["text"]["text"])


updated_variants = existing_prompt["variants"]
for variant in updated_variants:
    if variant["templateType"] == "TEXT":
        variant["templateConfiguration"]["text"]["text"] = "# HR Assistant: Job Description Generator\n\n## Task\nCreate a professional, inclusive job description based on the provided information.\n\n## Input Information\n### Job Title\n{{job_title}}\n\n### Responsibilities\n{{responsibilities}}\n\n### Requirements\n{{requirements}}\n\n### Location\n{{location}}\n\n### Work Type\n{{work_type}}\n\n## Output Requirements\n1. Begin with a clear, compelling summary of the position (1-2 sentences)\n2. Structure the job description with appropriate headings (About the Role, Responsibilities, Requirements, etc.)\n3. Use gender-neutral, inclusive language throughout\n4. Avoid unnecessary jargon or buzzwords\n5. Emphasize both skills and potential cultural fit\n6. Include location and work arrangement details clearly\n7. Keep the entire description under 250 words\n8. Ensure the tone is professional yet welcoming\n\nFormat your response as a complete job description without any explanations or additional commentary. Focus on creating content that will attract diverse, qualified candidates."



response = bedrock_agent.update_prompt(
    promptIdentifier=prompt_identifier,
    name=existing_prompt["name"],
    description=existing_prompt["description"],
    defaultVariant=existing_prompt["defaultVariant"],
    variants=updated_variants
)

print("\n==================== UPDATED PROMPT ====================\n")

updated_prompt = bedrock_agent.get_prompt(promptIdentifier=prompt_identifier)
print(updated_prompt["variants"][0]["templateConfiguration"]["text"]["text"])


==================== ORIGINAL PROMPT ====================


You are an HR assistant.

Write a professional, inclusive job description using the following inputs:

Job title: {{job_title}}
Responsibilities: {{responsibilities}}
Requirements: {{requirements}}
Location: {{location}}
Work type: {{work_type}}

- Start with a clear summary
- Use concise, inclusive language
- Keep it under 250 words


==================== UPDATED PROMPT ====================

# HR Assistant: Job Description Generator

## Task
Create a professional, inclusive job description based on the provided information.

## Input Information
### Job Title
{{job_title}}

### Responsibilities
{{responsibilities}}

### Requirements
{{requirements}}

### Location
{{location}}

### Work Type
{{work_type}}

## Output Requirements
1. Begin with a clear, compelling summary of the position (1-2 sentences)
2. Structure the job description with appropriate headings (About the Role, Responsibilities, Requirements, etc.)
3. Use gende

In [59]:
response = bedrock_agent.create_prompt_version(
    description='Optimized prompt for creating job description documents.',
    promptIdentifier=prompt_identifier
)

print("\n==================== PROMPT VERSION ARN ====================\n")
print(response["arn"])


==================== PROMPT VERSION ARN ====================

arn:aws:bedrock:us-east-1:982248023567:prompt/ARZ39REHBM:1


In [60]:
prompt_arn = "arn:aws:bedrock:us-east-1:982248023567:prompt/ARZ39REHBM:1"

response = bedrock_runtime.converse(
    modelId=prompt_arn,
    promptVariables={
        "job_title": {"text": "UX Designer"},
        "responsibilities": {"text": "Design user interfaces, run usability testing, collaborate with product teams"},
        "requirements": {"text": "3+ years experience, Figma, HTML/CSS knowledge, communication skills"},
        "location": {"text": "New York or remote"},
        "work_type": {"text": "Full-time"}
    }
)

print("\\n==================== Response Text ====================\\n")
print(response['output']['message']['content'][0]['text'])

\n==================== Response Text ====================\n
**UX Designer**

**About the Role**
Join our dynamic team as a UX Designer to create intuitive and engaging user interfaces. We are looking for a creative professional to run usability testing and collaborate with product teams to enhance user experience.

**Responsibilities**
- Design user interfaces that meet user needs
- Conduct usability testing to gather feedback
- Collaborate with product teams to implement design solutions

**Requirements**
- 3+ years of UX design experience
- Proficiency in Figma, HTML/CSS
- Strong communication skills
- Ability to work both independently and as part of a team

**Location**
New York or remote

**Work Type**
Full-time

We value diverse perspectives and are committed to creating an inclusive environment for all employees. If you are passionate about user experience and have the required skills, we would love to hear from you.


In [ ]:
prompt_arn = "<FILL_ME_IN>"

response = bedrock_runtime.converse(
    modelId=prompt_arn,
    promptVariables={
        "job_title": {"text": "UX Designer"},
        "responsibilities": {"text": "Design user interfaces, run usability testing, collaborate with product teams"},
        "requirements": {"text": "3+ years experience, Figma, HTML/CSS knowledge, communication skills"},
        "location": {"text": "New York or remote"},
        "work_type": {"text": "Full-time"}
    }
)

print("\\n==================== Response Text ====================\\n")
print(response['output']['message']['content'][0]['text'])

In [61]:
prompt_arn = "arn:aws:bedrock:us-east-1:982248023567:prompt/MI1S36L51W:2"

response = bedrock_runtime.converse(
    modelId=prompt_arn,
    promptVariables={
        "job_title": {"text": "UX Designer"},
        "responsibilities": {"text": "Design user interfaces, run usability testing, collaborate with product teams"},
        "requirements": {"text": "3+ years experience, Figma, HTML/CSS knowledge, communication skills"},
        "location": {"text": "New York or remote"},
        "work_type": {"text": "Full-time"}
    }
)

print("\\n==================== Response Text ====================\\n")
print(response['output']['message']['content'][0]['text'])

\n==================== Response Text ====================\n
**Job Title: UX Designer**

**Summary:**
We are seeking a talented and inclusive UX Designer to join our dynamic team in New York or remotely. As a UX Designer, you will be responsible for designing intuitive user interfaces, conducting usability testing, and collaborating closely with our product teams to enhance user experience. Your expertise in Figma, HTML/CSS, and strong communication skills will be pivotal in shaping our products.

**Responsibilities:**
- Design compelling and user-friendly interfaces.
- Conduct thorough usability testing to refine and improve user experience.
- Collaborate with cross-functional teams to integrate user feedback and insights.
- Utilize Figma for prototyping and design iterations.
- Apply HTML/CSS knowledge to create and implement design solutions.

**Requirements:**
- 3+ years of experience in UX design.
- Proficiency in Figma for design and prototyping.
- Strong understanding of HTML/CSS